In [1]:
%cd ../search/search
#! python pacman.py --layout originalClassic

C:\Users\Gabriel de Freitas\OneDrive\Documentos\Tarefas\Universidade\MC886\projects_MachineLearning\project3_ra214129_ra216179\search\search


In [3]:
from pacman import runGames, readCommand
from game import Agent, Directions
from util import manhattanDistance
import numpy as np
import random
import json

In [4]:
random.seed(7)

In [12]:
def getState(state):
    currentState = (state.getPacmanPosition(),Qlearn.getNearestFoodDis(state),state.getNumFood() + len(state.getCapsules()),Qlearn.nearestGhostPos(state))
    return currentState

In [39]:
class Qlearn(Agent):
    def __init__(self, alpha,gamma,epsilon = 1,Qtable = {}):
        self.Qtable = Qtable
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.score = 0
        self.action = None
        self.currentState = None
    
    def getQvalue(self,state,action):
        pair = str((state,action))
        return self.Qtable.get(pair,0.0)
    
    def getAction(self, state):
        actions = state.getLegalPacmanActions()
        currentState = getState(state)
        action = self.getBestAction(currentState,actions)
        state2 = state.generatePacmanSuccessor(action)
        nextState = getState(state2)
        if state2.isLose():
            reinforcement = -20000
        elif state2.isWin():
            reinforcement = 20000
        else:
            reinforcement = self.getReward(currentState,nextState,state2.getScore())
        self.learn(currentState,nextState,action,actions,reinforcement)
        self.action = action
        self.currentState = currentState
        return action
        
        
    def nearestGhostPos(state):
        ghostList = state.getGhostPositions()
        nearestDistance = Qlearn.getNearGhost(state.getPacmanPosition(),ghostList)
        return nearestDistance

    def getNearGhost(pacman,ghosts):
        minDis = float("inf")
        minPos = -1
        for i in range(len(ghosts)):
            distance = manhattanDistance(pacman,ghosts[i])
            if distance < minDis:
                minDis = distance
                minPos = i
        if minPos > -1:
            return ghosts[minPos]
        else:
            return (-1,-1)

    def getNearestFoodDis(state):
        posList = Qlearn.getFoodPos(state.getFood())
        posList = posList + state.getCapsules()
        minDis = minDis = float("inf")
        pacPos = state.getPacmanPosition()
        for i in posList:
            distance = manhattanDistance(pacPos,i)
            if distance < minDis:
                minDis = distance
        return minDis


    def getFoodPos(grid):
        posList = []
        gridList = grid.asList()
        for i in range(len(gridList)):
            for j in range(len(gridList[i])):
                if gridList[i][j]:
                    posList.append((i,j))
        return posList

    def getBestAction(self,currentState,actions):
        if random.random() < self.epsilon:
            return random.choice(actions)
        maxValue = float("-inf")
        maxAction = []
        for i in actions:
            value = self.getQvalue(currentState,i)
            if value > maxValue:
                maxValue = value
                maxAction = [i]
            if value == maxValue:
                maxAction.append(i)
        #print(maxValue)
        #print(maxAction)
        return random.choice(maxAction)

    def learn(self,currentState,nextState,action,actions,reinforcement):
        nextActions = [self.getQvalue(nextState,a) for a in actions]
        if len(nextActions) > 0:
            newQ = max(nextActions)
        else:
            newQ = 0
        currentQ = self.getQvalue(currentState,action)
        self.Qtable[str((currentState,action))] = currentQ + self.alpha * (reinforcement + (self.gamma*newQ) - currentQ)

    def setEpsilon(self,epsilon):
        self.epsilon = epsilon

    def serialize(self,path):
        try:
            f = open(path,"w")
            json.dump(self.Qtable,f)
            f.close()
        except:
            raise Exception("You've got an error")

    def desserialize(self,path):
        try:
            f = open(path,"r")
            self.Qtable = json.load(f)
            f.close()
        except:
            raise Exception("You've got an error")
    
    def getReward(self,state1,state2,score):
        pacGhostDistance1 = manhattanDistance(state1[0],state1[3])
        pacGhostDistance2 = manhattanDistance(state2[0],state2[3])
        eatenFood = state2[2] - state1[2]
        ghostDistance = pacGhostDistance2 - pacGhostDistance1
        foodDistance = state2[1] - state1[1]
        if ghostDistance > 0:
            ghostReward = ghostDistance*100
        else:
            ghostReward = 50*ghostDistance
        if foodDistance < 0:
            foodReward = -100 * foodDistance
        else:
            foodReward = 0
        if eatenFood > 0:
            eatReward = 200*eatenFood
        else:
            eatReward = 0
        scoreReward = 0.5*(score - self.score)
        return ghostReward + foodReward + eatReward + scoreReward
            

In [25]:
agente = Qlearn(0.75,0.4,epsilon = 1)
args = readCommand(["-q", "--pacman", "LeftTurnAgent","--layout","smallClassic"])
args["pacman"] = agente

['-q', '--pacman', 'LeftTurnAgent', '--layout', 'smallClassic']


In [26]:
a = runGames(**args)
print(a)

In [41]:
args = readCommand(["-q", "--pacman", "LeftTurnAgent","--layout","smallClassic"])

['-q', '--pacman', 'LeftTurnAgent', '--layout', 'smallClassic']


In [43]:
a = []

In [45]:
agente = Qlearn(0.8,0.4,epsilon = 1)
args["pacman"] = agente

In [ ]:
for i in range(30000):
    agente.setEpsilon(1/(i+1))
    a.append(runGames(**args))
    if a[i][0].state.isLose():
        agente.learn(agente.currentState,getState(a[i][0].state),agente.action,a[i][0].state.getLegalPacmanActions(),-20000)
    elif a[i][0].state.isWin():
        agente.learn(agente.currentState,getState(a[i][0].state),agente.action,a[i][0].state.getLegalPacmanActions(),20000)
        print(i)

In [ ]:
print(a[len(a)-1][0].state.isWin())
print(a[len(a)-1][0].state.getScore())

In [ ]:
agente.serialize("../../notebooks/results/a075-g04-d040721.json")

In [28]:
print(a[0].state.isWin())

False
